从生成的插补文件，与fuse的pkl合并起来，根据stay_id对齐，代替其中的reg_ts

In [ ]:
# 取中位数
# 转换插补后的pkl 并且取中位数后在观测点覆盖真实值

import pickle
import numpy as np
import torch

INPUT_PKL = "/playpen-shared/kechengli/workspace/Fusemoe/CSDI-main/save/ihm_fold_train10_20251119_004830/generated_outputs_nsample30.pk"
OUTPUT_PKL = "/playpen-shared/kechengli/workspace/Fusemoe/CSDI-main/save/ihm_fold_train10_20251119_004830/forFuse_train01.pk"


def to_np(x):
    if isinstance(x, torch.Tensor):
        return x.detach().cpu().numpy()
    return np.asarray(x)


print(f"📦 加载原始 PKL: {INPUT_PKL}")

with open(INPUT_PKL, "rb") as f:
    data = pickle.load(f)

(
    all_generated_samples,   # (N, nsample, L, K)
    all_target,              # (N, L, K)
    all_evalpoint,           # (N, L, K)
    all_observed_point,      # (N, L, K)
    all_observed_time,       # (N, L)
    scaler,
    mean_scaler,
    patient_ids              # list[str]
) = data

# 转 numpy
gen = to_np(all_generated_samples)   # (N,nsample,L,K)
target = to_np(all_target)           # (N,L,K)
obsmask = to_np(all_observed_point)  # (N,L,K)
patient_ids = list(patient_ids)

N, nsample, L, K = gen.shape
print(f"生成样本 gen: {gen.shape}, target: {target.shape}, obsmask: {obsmask.shape}")

# ---------------------------------------------------------
# ⭐ Step 1: 对 nsample 维做平均 → (N,L,K)
# ---------------------------------------------------------
print("➡ 计算 nsample 维度上的中位数...")
gen_median = np.median(gen, axis=1)   # (N, L, K)

# ---------------------------------------------------------
# ⭐ Step 2: 在观测点用 target 覆盖 gen_mean
# obsmask == 1 的位置：用真实 target 替换模型生成值
# ---------------------------------------------------------

print("➡ 在观测点用 target 覆盖生成结果...")
new_generated = gen_median.copy()
new_generated[obsmask == 1] = target[obsmask == 1]

# ---------------------------------------------------------
# ⭐ Step 3: 保存新的 PKL 文件
# 只包含：new_generated, obsmask, patient_ids
# ---------------------------------------------------------

print(f"➡ 保存处理后的 PKL 到: {OUTPUT_PKL}")

with open(OUTPUT_PKL, "wb") as f:
    pickle.dump(
        [
            new_generated,    # (N,48,30)
            obsmask,          # (N,48,30)
            patient_ids       # list[str]
        ],
        f
    )

print("🎉 完成！输出内容：")
print(" new_generated shape =", new_generated.shape)
print(" obsmask shape       =", obsmask.shape)
print(" patient_ids count   =", len(patient_ids))


In [1]:
# 检查字段

#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import pickle
import numpy as np
import torch
import pandas as pd
from pprint import pprint

# 你的 pkl 路径
PK_PATH = "/playpen-shared/kechengli/workspace/Fusemoe/CSDI-main/save/ihm_fold_val10_20251119_004936/generated_outputs_nsample30.pk"

# 要查看的样本编号
SAMPLE_IDX = 0


def to_numpy(x):
    """兼容 numpy / torch 张量"""
    if isinstance(x, torch.Tensor):
        return x.detach().cpu().numpy()
    return np.asarray(x)


def preview_matrix(x, r=5, c=6):
    """裁取矩阵前几行列方便预览"""
    x = to_numpy(x)
    L, K = x.shape[:2]
    r = min(r, L)
    c = min(c, K)
    return pd.DataFrame(
        x[:r, :c],
        index=[f"t{i}" for i in range(r)],
        columns=[f"f{j}" for j in range(c)]
    )


def main():
    print(f"📦 正在加载: {PK_PATH}")

    with open(PK_PATH, "rb") as f:
        data = pickle.load(f)

    print("\n=== 文件结构 ===")
    print("类型:", type(data))
    if isinstance(data, (list, tuple)):
        print("元素个数:", len(data))
    print("-" * 60)

    # 按照 evaluate() 的保存顺序解包
    (
        all_generated_samples,   # 0
        all_target,              # 1
        all_evalpoint,           # 2
        all_observed_point,      # 3
        all_observed_time,       # 4
        scaler,                  # 5
        mean_scaler,             # 6
        patient_ids              # 7
    ) = data

    # 转 numpy
    gen = to_numpy(all_generated_samples)
    target = to_numpy(all_target)
    evalmask = to_numpy(all_evalpoint)
    obsmask = to_numpy(all_observed_point)
    times = to_numpy(all_observed_time)

    print("all_generated_samples :", type(all_generated_samples), gen.shape)
    print("all_target            :", type(all_target), target.shape)
    print("all_evalpoint         :", type(all_evalpoint), evalmask.shape)
    print("all_observed_point    :", type(all_observed_point), obsmask.shape)
    print("all_observed_time     :", type(all_observed_time), times.shape)
    print("scaler                :", type(scaler))
    print("mean_scaler           :", type(mean_scaler))
    print("patient_ids (#)       :", len(patient_ids))
    print("-" * 60)

    # ---- 选择一个样本查看 ----
    idx = SAMPLE_IDX
    print(f"\n📌 查看样本 index = {idx}, 对应 patient_id = {patient_ids[idx]}")
    print(f"原始 txt: ./data/ihm_data_train/{patient_ids[idx]}.txt")

    N, nsample, L, K = gen.shape

    print("\n=== 1) 插补生成值示例 (gen) ===")
    print("shape:", gen[idx].shape)
    print("前几行前几列:")
    print(preview_matrix(gen[idx, 0]))   # 使用第 0 条采样轨迹

    print("\n=== 2) 目标真值 (target) ===")
    print("shape:", target[idx].shape)
    print(preview_matrix(target[idx]))

    print("\n=== 3) 评测掩码 evalmask (1=评测点) ===")
    print("shape:", evalmask[idx].shape)
    print(preview_matrix(evalmask[idx]))

    print("\n=== 4) 观测掩码 obsmask (1=原始观测点) ===")
    print("shape:", obsmask[idx].shape)
    print(preview_matrix(obsmask[idx]))

    print("\n=== 5) 时间索引 times ===")
    print("shape:", times[idx].shape)
    print("前 20 个:", times[idx][:20])

    print("\n=== Summary ===")
    summary = {
        "generated_samples_shape": gen.shape,
        "target_shape": target.shape,
        "evalmask_shape": evalmask.shape,
        "obsmask_shape": obsmask.shape,
        "times_shape": times.shape,
        "num_samples": N,
        "nsample_per_series": nsample,
        "sequence_length": L,
        "num_features": K,
        "example_patient_id": patient_ids[idx],
    }
    pprint(summary)


if __name__ == "__main__":
    main()


📦 正在加载: /playpen-shared/kechengli/workspace/Fusemoe/CSDI-main/save/ihm_fold_val10_20251119_004936/generated_outputs_nsample30.pk

=== 文件结构 ===
类型: <class 'list'>
元素个数: 8
------------------------------------------------------------
all_generated_samples : <class 'torch.Tensor'> (5270, 30, 48, 30)
all_target            : <class 'torch.Tensor'> (5270, 48, 30)
all_evalpoint         : <class 'torch.Tensor'> (5270, 48, 30)
all_observed_point    : <class 'torch.Tensor'> (5270, 48, 30)
all_observed_time     : <class 'torch.Tensor'> (5270, 48)
scaler                : <class 'int'>
mean_scaler           : <class 'int'>
patient_ids (#)       : 5270
------------------------------------------------------------

📌 查看样本 index = 0, 对应 patient_id = 30004144
原始 txt: ./data/ihm_data_train/30004144.txt

=== 1) 插补生成值示例 (gen) ===
shape: (30, 48, 30)
前几行前几列:
          f0        f1        f2        f3        f4        f5
t0  0.131641 -0.527381  1.000000 -0.104887 -0.002216 -0.016950
t1  0.016158 -0.523586  1.

In [ ]:
# 中位数代码取代观测点真实值

In [2]:
# 中位数后的

#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import pickle
import numpy as np
import pandas as pd

# 修改成你的实际路径
PK_PATH = "/playpen-shared/kechengli/workspace/Fusemoe/CSDI-main/save/ihm_fold_val10_20251119_004936/forFuse_val01.pk"


def show_matrix(name, mat, rows=10, cols=10):
    """格式化显示矩阵前几行几列"""
    mat = np.array(mat)
    r = min(rows, mat.shape[0])
    c = min(cols, mat.shape[1])
    df = pd.DataFrame(mat[:r, :c])
    print(f"\n{name} (前 {r} 行 × 前 {c} 列):")
    print(df)


def main():
    print(f"📦 正在加载: {PK_PATH}")
    with open(PK_PATH, "rb") as f:
        new_generated, obsmask, patient_ids = pickle.load(f)

    print("=== 文件已加载 ===")
    print("new_generated shape:", new_generated.shape)
    print("obsmask shape:", obsmask.shape)
    print("patient_ids 数量:", len(patient_ids))

    N = new_generated.shape[0]

    # 打印前三个样本
    for idx in range(min(3, N)):
        print("\n" + "=" * 60)
        print(f"样本 {idx}")
        print(f"patient_id: {patient_ids[idx]}")
        print(f"生成矩阵 shape: {new_generated[idx].shape}")
        print(f"观测掩码 shape: {obsmask[idx].shape}")

        # 显示前几行×前几列
        show_matrix("生成矩阵 new_generated", new_generated[idx], rows=10, cols=10)
        show_matrix("观测掩码 obsmask", obsmask[idx], rows=10, cols=10)


if __name__ == "__main__":
    main()


📦 正在加载: /playpen-shared/kechengli/workspace/Fusemoe/CSDI-main/save/ihm_fold_val10_20251119_004936/forFuse_val01.pk
=== 文件已加载 ===
new_generated shape: (5270, 48, 30)
obsmask shape: (5270, 48, 30)
patient_ids 数量: 5270

样本 0
patient_id: 30004144
生成矩阵 shape: (48, 30)
观测掩码 shape: (48, 30)

生成矩阵 new_generated (前 10 行 × 前 10 列):
          0         1         2         3         4         5         6  \
0 -0.656078 -0.449664  0.559013 -0.050257 -0.045637 -0.013254 -0.007635   
1 -0.714645 -0.520701  0.062383 -0.052153 -0.048383 -0.011813 -0.006888   
2 -0.685689 -0.503291  0.228359 -0.034169 -0.047242 -0.004162 -0.010574   
3 -0.649891 -0.508481  0.226019 -0.032072 -0.047014  0.001198 -0.009326   
4 -0.668029 -0.550761  0.393447 -0.037966 -0.032234 -0.007022 -0.007772   
5 -0.705800 -0.491557  0.228101 -0.014448 -0.045190 -0.006805 -0.009892   
6 -0.639427 -0.571216  0.228479 -0.027025 -0.043435  0.000378 -0.009361   
7 -0.630678 -0.554704  0.227019 -0.009485 -0.045403 -0.001369 -0.008324   
8

In [4]:
# 合并pkl生成新的
# %% 导入依赖
import os
import pickle
import numpy as np

# 基础路径
base_csdi = "/playpen-shared/kechengli/workspace/Fusemoe/CSDI-main/save"
base_stays = "/playpen-shared/kechengli/workspace/dataset/mimiciv_pkl"
output_root = "/playpen-shared/kechengli/workspace/dataset/mimic_after_diff_csdi2"

os.makedirs(output_root, exist_ok=True)

# 需要循环处理的三个 split
configs = [
    {
        "name": "test",
        "csdi_pkl": f"{base_csdi}/ihm_fold_test10_20251119_004908/forFuse_test01.pk",
        "stays_pkl": f"{base_stays}/test_ihm-48-cxr-notes-ecg-missingInd_stays.pkl",
    },
    {
        "name": "val",
        "csdi_pkl": f"{base_csdi}/ihm_fold_val10_20251119_004936/forFuse_val01.pk",
        "stays_pkl": f"{base_stays}/val_ihm-48-cxr-notes-ecg-missingInd_stays.pkl",
    },
    {
        "name": "train",
        "csdi_pkl": f"{base_csdi}/ihm_fold_train10_20251119_004830/forFuse_train01.pk",
        "stays_pkl": f"{base_stays}/train_ihm-48-cxr-notes-ecg-missingInd_stays.pkl",
    },
]

# %% 定义工具函数

def merge_csdi_into_stays(csdi_pkl_path, stays_pkl_path, output_root):
    """
    读取 CSDI forFuse_xxx.pk 和 对应的 *_stays.pkl，
    将 reg_ts 替换为 new_generated，将 reg_ts_mask 替换为 obsmask，
    按 stay_id ↔ patient_id 对齐，输出到 output_root 下同名文件。
    """
    print("=" * 80)
    print(f"📦 合并:\n  CSDI:  {csdi_pkl_path}\n  STAYS: {stays_pkl_path}")

    # ---- 加载 stays 数据 ----
    with open(stays_pkl_path, "rb") as f:
        stays_data = pickle.load(f)   # list[dict]
    print(f"原始 stays 样本数: {len(stays_data)}")

    # ---- 加载 CSDI 输出 ----
    with open(csdi_pkl_path, "rb") as f:
        new_generated, obsmask, patient_ids = pickle.load(f)

    new_generated = np.asarray(new_generated)  # (N, 48, 30)
    obsmask = np.asarray(obsmask)              # (N, 48, 30)
    patient_ids = list(patient_ids)

    print(f"CSDI 样本数: {len(patient_ids)}")
    print("new_generated shape:", new_generated.shape)
    print("obsmask shape      :", obsmask.shape)

    # ---- 建立 patient_id → index 映射 ----
    pid2idx = {str(pid): i for i, pid in enumerate(patient_ids)}

    missing_count = 0
    updated_count = 0

    # ---- 遍历每个 stay 样本，按 stay_id 匹配 ----
    for i, item in enumerate(stays_data):
        stay_id = str(item["stay_id"])  # stay_id 是 int，这里转 str 与 patient_ids 对齐

        if stay_id not in pid2idx:
            missing_count += 1
            continue

        j = pid2idx[stay_id]

        reg_ts_new = new_generated[j]   # (48, K)
        mask_new = obsmask[j]           # (48, K)

        old_reg_ts = item.get("reg_ts", None)
        if old_reg_ts is None:
            # 如果原本没有 reg_ts 字段，就直接赋值
            item["reg_ts"] = reg_ts_new
            item["reg_ts_mask"] = mask_new
            updated_count += 1
            continue

        old_reg_ts = np.asarray(old_reg_ts)
        old_shape = old_reg_ts.shape

        # ---- 简单长度检查：理想情况应该是 (48, K) ----
        if old_shape[0] != reg_ts_new.shape[0]:
       # 直接用新插补结果覆盖
            item["reg_ts"] = reg_ts_new
            item["reg_ts_mask"] = mask_new
        else:
            # 正常替换
            item["reg_ts"] = reg_ts_new
            item["reg_ts_mask"] = mask_new

        updated_count += 1

    print(f"✅ 匹配并更新的样本数: {updated_count}")
    print(f"⚠️ CSDI 中不存在的样本数: {missing_count}")

    # ---- 保存到新的目录，文件名与原 stays_pkl 一致 ----
    basename = os.path.basename(stays_pkl_path)
    out_path = os.path.join(output_root, basename)

    print(f"💾 保存到: {out_path}")
    with open(out_path, "wb") as f:
        pickle.dump(stays_data, f)

    print("🎉 完成该 split 合并\n")
    return out_path

# %% 循环处理 train / val / test

output_paths = []

for cfg in configs:
    print(f"\n🚀 处理中 split = {cfg['name']}")
    out_path = merge_csdi_into_stays(
        csdi_pkl_path=cfg["csdi_pkl"],
        stays_pkl_path=cfg["stays_pkl"],
        output_root=output_root,
    )
    output_paths.append((cfg["name"], out_path))

print("\n✅ 所有 split 处理完成，输出文件列表：")
for name, path in output_paths:
    print(f"{name}: {path}")



🚀 处理中 split = test
📦 合并:
  CSDI:  /playpen-shared/kechengli/workspace/Fusemoe/CSDI-main/save/ihm_fold_test10_20251119_004908/forFuse_test01.pk
  STAYS: /playpen-shared/kechengli/workspace/dataset/mimiciv_pkl/test_ihm-48-cxr-notes-ecg-missingInd_stays.pkl
原始 stays 样本数: 5269
CSDI 样本数: 5269
new_generated shape: (5269, 48, 30)
obsmask shape      : (5269, 48, 30)
✅ 匹配并更新的样本数: 5269
⚠️ CSDI 中不存在的样本数: 0
💾 保存到: /playpen-shared/kechengli/workspace/dataset/mimic_after_diff_csdi2/test_ihm-48-cxr-notes-ecg-missingInd_stays.pkl
🎉 完成该 split 合并


🚀 处理中 split = val
📦 合并:
  CSDI:  /playpen-shared/kechengli/workspace/Fusemoe/CSDI-main/save/ihm_fold_val10_20251119_004936/forFuse_val01.pk
  STAYS: /playpen-shared/kechengli/workspace/dataset/mimiciv_pkl/val_ihm-48-cxr-notes-ecg-missingInd_stays.pkl
原始 stays 样本数: 5270
CSDI 样本数: 5270
new_generated shape: (5270, 48, 30)
obsmask shape      : (5270, 48, 30)
✅ 匹配并更新的样本数: 5270
⚠️ CSDI 中不存在的样本数: 0
💾 保存到: /playpen-shared/kechengli/workspace/dataset/mimic_after_diff

In [6]:
# check 新数据
import pickle
import numpy as np
import pandas as pd

PKL_PATH = "/playpen-shared/kechengli/workspace/dataset/mimic_after_diff_csdi2/val_ihm-48-cxr-notes-ecg-missingInd_stays.pkl"

print(f"📦 正在加载: {PKL_PATH}")

with open(PKL_PATH, "rb") as f:
    data = pickle.load(f)

print(f"✅ 加载成功，共 {len(data)} 条样本")
print("类型:", type(data))

# -------------------------------
# 基础字段统计
# -------------------------------

all_keys = set()
for item in data:
    all_keys.update(item.keys())

print("\n📌 数据集中所有字段:")
print(sorted(list(all_keys)))

# -------------------------------
# 字段维度统计
# -------------------------------

def print_shape(name, arr):
    try:
        arr = np.asarray(arr)
        print(f"{name:20s} shape = {arr.shape}")
    except Exception as e:
        print(f"{name:20s} 无法转换为 array ({e})")

print("\n📌 检查前 3 条样本的字段与形状:")
for i in range(min(3, len(data))):
    item = data[i]
    print(f"\n================= 样本 {i} (stay_id={item['stay_id']}) =================")
    for key, val in item.items():
        if isinstance(val, np.ndarray) or isinstance(val, list):
            print_shape(key, val)
        else:
            print(f"{key:20s} type={type(val)}  value={val}")

# -------------------------------
# 新增 reg_ts / reg_ts_mask 校验
# -------------------------------

print("\n📌 检查 reg_ts 和 reg_ts_mask 是否已替换为 (48,30):")

incorrect_count = 0
for i, item in enumerate(data):
    if "reg_ts" not in item or "reg_ts_mask" not in item:
        print(f"⚠️ 样本 {i} 缺失 reg_ts 或 reg_ts_mask")
        incorrect_count += 1
        continue

    reg_ts = np.asarray(item["reg_ts"])
    reg_ts_mask = np.asarray(item["reg_ts_mask"])

    if reg_ts.shape != (48, 30) or reg_ts_mask.shape != (48, 30):
        print(f"❌ 样本 {i} reg_ts={reg_ts.shape}, mask={reg_ts_mask.shape}")
        incorrect_count += 1

if incorrect_count == 0:
    print("🎉 所有 reg_ts / reg_ts_mask 形状正确 (48,30)！")
else:
    print(f"⚠️ 有 {incorrect_count} 个样本 reg_ts 形状不匹配")

# -------------------------------
# 数值范围检查
# -------------------------------

print("\n📌 生成值范围检查 (reg_ts):")
all_vals = np.concatenate([np.asarray(item["reg_ts"]).reshape(-1) for item in data])
print("min =", np.nanmin(all_vals))
print("max =", np.nanmax(all_vals))

# -------------------------------
# 掩码检查
# -------------------------------

print("\n📌 掩码值检查 (reg_ts_mask):")
all_masks = np.concatenate([np.asarray(item["reg_ts_mask"]).reshape(-1) for item in data])
unique_masks = np.unique(all_masks)
print("掩码唯一值:", unique_masks)

if set(unique_masks) <= {0,1}:
    print("🎉 掩码正确，只有 0/1")
else:
    print("❌ 掩码存在异常值:", unique_masks)


📦 正在加载: /playpen-shared/kechengli/workspace/dataset/mimic_after_diff_csdi2/val_ihm-48-cxr-notes-ecg-missingInd_stays.pkl
✅ 加载成功，共 5270 条样本
类型: <class 'list'>

📌 数据集中所有字段:
['cxr_feats', 'cxr_missing', 'cxr_time', 'ecg_feats', 'ecg_missing', 'ecg_time', 'hadm_id', 'irg_ts', 'irg_ts_mask', 'label', 'name', 'reg_ts', 'reg_ts_mask', 'stay_id', 'text_data', 'text_embeddings', 'text_missing', 'text_time_to_end', 'ts_tt']

📌 检查前 3 条样本的字段与形状:

================= 样本 0 (stay_id=36850695) =================
name                 type=<class 'numpy.int64'>  value=14120635
hadm_id              type=<class 'numpy.int64'>  value=26800029
stay_id              type=<class 'int'>  value=36850695
ts_tt                shape = (110,)
irg_ts               shape = (110, 30)
irg_ts_mask          shape = (110, 30)
reg_ts               shape = (48, 30)
text_data            shape = (1,)
text_time_to_end     shape = (1,)
text_embeddings      shape = (1, 768)
text_missing         type=<class 'int'>  value=0
cxr_feats 

In [ ]:
# 切换版本 MulEHR环境跑
import os
import sys
import pickle
import types
import numpy as np
import importlib

# --- 关键：安装 numpy2 → numpy1 的 pickle 兼容补丁 ---
import sys
import types
import numpy as np

def install_numpy2_pickle_shim():
    """
    在当前 Python 进程里伪造 numpy 2.x 的模块路径结构：
    - numpy._core           （包）
    - numpy._core.<submod>  映射到 numpy.core.<submod>
    这样可以在 numpy 1.x 环境里反序列化 numpy 2.x 生成的 pickle。
    """
    # 如果已经装过就不重复
    if any(name.startswith("numpy._core.numeric") for name in sys.modules):
        return

    # 1) 创建 numpy._core 包
    core_pkg_name = "numpy._core"
    core_pkg = sys.modules.get(core_pkg_name)
    if core_pkg is None:
        core_pkg = types.ModuleType(core_pkg_name)
        core_pkg.__path__ = []      # 标记为包
        sys.modules[core_pkg_name] = core_pkg

    # 2) 映射常见子模块
    submodules = [
        "multiarray",
        "_multiarray_umath",
        "numeric",
        "numerictypes",
        "umath",
        "fromnumeric",
        "function_base",
        "shape_base",
        "getlimits",
        "machar",
    ]

    for name in submodules:
        full_old = f"numpy._core.{name}"
        full_new = f"numpy.core.{name}"
        if full_old in sys.modules:
            continue
        try:
            real_mod = importlib.import_module(full_new)
        except ImportError:
            # 某些 numpy 版本可能没有某个子模块，忽略就行
            continue

        # 把真实模块挂到假的路径上
        sys.modules[full_old] = real_mod
        # 同时作为 numpy._core 的属性
        setattr(core_pkg, name, real_mod)


def convert_one(src_path, dst_path):
    print(f"[LOAD ] {src_path}")
    install_numpy2_pickle_shim()  # 先装补丁，再读

    with open(src_path, "rb") as f:
        obj = pickle.load(f)

    os.makedirs(os.path.dirname(dst_path), exist_ok=True)
    # 用 protocol=4 存，确保 Py3.8/3.9 + numpy1.x 兼容
    with open(dst_path, "wb") as f:
        pickle.dump(obj, f, protocol=4)

    print(f"[SAVE ] {dst_path}  (protocol=4)")


def main():
    in_dir  = "/playpen-shared/kechengli/workspace/dataset/mimic_after_diff_csdi_2"
    out_dir = "/playpen-shared/kechengli/workspace/dataset/mimiciv_after_diff_fuse_2"

    files = [
        "train_ihm-48-cxr-notes-ecg-missingInd_stays.pkl",
        "val_ihm-48-cxr-notes-ecg-missingInd_stays.pkl",
        "test_ihm-48-cxr-notes-ecg-missingInd_stays.pkl",
    ]

    for fname in files:
        src = os.path.join(in_dir, fname)
        dst = os.path.join(out_dir, fname)
        convert_one(src, dst)

    print("\nAll done. Now use the files in:", out_dir)



main()


[LOAD ] /playpen-shared/kechengli/workspace/dataset/mimic_after_diff_csdi_2/train_ihm-48-cxr-notes-ecg-missingInd_stays.pkl
[SAVE ] /playpen-shared/kechengli/workspace/dataset/mimiciv_after_diff_fuse_2/train_ihm-48-cxr-notes-ecg-missingInd_stays.pkl  (protocol=4)
[LOAD ] /playpen-shared/kechengli/workspace/dataset/mimic_after_diff_csdi_2/val_ihm-48-cxr-notes-ecg-missingInd_stays.pkl
[SAVE ] /playpen-shared/kechengli/workspace/dataset/mimiciv_after_diff_fuse_2/val_ihm-48-cxr-notes-ecg-missingInd_stays.pkl  (protocol=4)
[LOAD ] /playpen-shared/kechengli/workspace/dataset/mimic_after_diff_csdi_2/test_ihm-48-cxr-notes-ecg-missingInd_stays.pkl
[SAVE ] /playpen-shared/kechengli/workspace/dataset/mimiciv_after_diff_fuse_2/test_ihm-48-cxr-notes-ecg-missingInd_stays.pkl  (protocol=4)

All done. Now use the files in: /playpen-shared/kechengli/workspace/dataset/mimiciv_after_diff_fuse_2
